# 0. Setup

In [ ]:
import nilearn
import pandas as pd
import numpy as np
import os

from nilearn import plotting
from nilearn import connectome
from nilearn import datasets
from nilearn import image



In [ ]:
# Set the path to the data directory
src_dir = '../data'

# Set the participant and session IDs
part_id = 'sub-01'
ses_id = 'ses-01'

# Path to the T1w image
anat_path = os.path.join(src_dir, part_id, ses_id, 'anat', f'{part_id}_{ses_id}_space-MNI152NLin2009cAsym_res-2_desc-preproc_T1w.nii.gz')

# Path to functional image
funcdir_path = os.path.join(src_dir, part_id, ses_id, 'func')

## 0.1 Anat

In [ ]:
from nilearn.plotting import plot_anat

plot_anat(anat_path, title='T1w image', display_mode='ortho',  draw_cross=False,  cut_coords=(0, 0, 0))


In [ ]:
# load the functional data

# list directory contents with extension 'nii.gz'
func_runs = os.listdir(funcdir_path)
func_runs = [f for f in func_runs if f.endswith('bold.nii.gz')]

# Sort the list
func_runs.sort()

print(func_runs)

In [ ]:
from nilearn.image import mean_img , load_img
from nilearn.plotting import plot_img

# Load the first functional image
func_image = image.load_img(os.path.join(funcdir_path,func_runs[0]))

mfunc_img = mean_img(func_image)

plot_img(mfunc_img)

# 1. Load Mask / atlas / meta

In [ ]:
# load atlas
from nilearn import datasets
atlas = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr50-2mm')
atlas_filename = atlas.maps

# list of labels
labels = atlas.labels

print(labels)

# number of regions
number_of_regions = len(labels[1:])
print(number_of_regions)

# plot the atlas
from nilearn import plotting
plotting.plot_roi(atlas_filename, title="Harvard Oxford atlas")

# 2. Extract timecourses per ROI / block

In [ ]:
# extract time series from ROIs
from nilearn.input_data import NiftiLabelsMasker

masker = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True, memory='nilearn_cache')

# time_series = masker.fit_transform(func_image)


In [ ]:
fdir_events = [file for file in os.listdir(funcdir_path) if file.endswith('.tsv')]

f_events = [os.path.join(funcdir_path, f) for f in fdir_events]

f_events.sort()

print(f_events)


# 3. Compute connectivity

In [ ]:
from nilearn.image import index_img 


onset_set = {}
duration_set = {}

target_set = {}

group_set = {}


offset_st = 4 # offset for the onset time
offset_end = 2 # offset for the end block time

# Compute the correlation matrix and store it in the partial correlation matrix as a 4D array
conn_measure = connectome.ConnectivityMeasure(kind='correlation',
    standardize="zscore_sample",)

# intialize 2D array to store the partial correlation matrix 
partial_correlation_matrix = {}

run_id = 0
idx = 0

# for each events file 
for fn in f_events:

    print(os.path.join(funcdir_path,func_runs[run_id]), fn)

    # Load the events.tsv file
    events = pd.read_table(fn)

    # Load the corresponding functional image
    func_image = image.load_img(os.path.join(funcdir_path,func_runs[run_id]))

    # extract time course from functional image
    time_series = masker.fit_transform(func_image)

    # create one image per event 
    for i, row in events.iterrows():

        # Print the row information
        # print(row) 

        # Extract the onset time
        onset = np.round(row['onset'])

        # Extract the duration
        duration = np.round(row['duration'])

        # Extract the trial_type
        trial_type = row['trial_type']

        # Print the row information
        print(onset, duration, trial_type) 

        # if duration greater than 10 and not 'Noise'
        if duration > 10 and onset + duration + offset_end < 660 and trial_type != 'Noise':

            onset_set[idx] = onset + offset_st

            duration_set[idx] = duration - offset_st + offset_end

            target_set[idx] = trial_type

            group_set[idx] = run_id

            # Extract the time series for the event
            event_time_series = time_series[int(onset):int(onset + duration), :]

            
            # 2D array for the correlation matrix
            temp_correlation_matrix = conn_measure.fit_transform([event_time_series])[0]

            # vectorize the correlation matrix
            partial_correlation_matrix[idx] = temp_correlation_matrix.flatten()

            idx += 1

    run_id += 1

## 3.1 Visualize average results per emotion

In [ ]:
# Visualize the average correlation matrix for each type of event

for target in set(target_set.values()):
    
    # print(target)

    # Normalize the correlation matrix

    # Get the number of events for the target
    counter = 0

    fl_correlation_matrix = np.zeros((number_of_regions * number_of_regions))

    for key, value in target_set.items():


        if value == target:
            fl_correlation_matrix += partial_correlation_matrix[key]
            counter += 1


    fl_correlation_matrix /= counter

    TwoD_correlation_matrix = fl_correlation_matrix.reshape(number_of_regions,number_of_regions)
    # Plot the correlation matrix
    plotting.plot_matrix(TwoD_correlation_matrix, labels=labels[1:], vmax=0.8, vmin=-0.8, title=target)


    # Save the correlation matrix as a nifti file
    correlation_matrix_img = masker.inverse_transform(TwoD_correlation_matrix)

    correlation_matrix_img.to_filename(f'correlation_matrix_{target}.nii.gz')



# 4. Compare emotions and save

In [ ]:

# two-factorial anova analysis
# one factor is the type of event
# the other factor is the region

# create a dataframe to store the correlation matrix
df = pd.DataFrame(partial_correlation_matrix.values())

# add the target column
df['target'] = target_set.values()




In [ ]:
# rename the columns
df.columns = [f'conn_{i}' for i in range(number_of_regions * number_of_regions)] + ['target']

In [ ]:
df

In [ ]:
# ANOVA analysis for each region

from statsmodels.formula.api import ols

from statsmodels.stats.anova import anova_lm

# create a dictionary to store the results
results = {}

df = df.rename(columns={0: "conn0"})

for i in range(number_of_regions * number_of_regions):
    
        # create the formula
        formula = f'conn_{i} ~ C(target)'
    
        # create the model
        model = ols(formula, data=df).fit()
    
        # perform the ANOVA
        aov_table = anova_lm(model, typ=2)
    
        # store the results
        results[i] = aov_table
    

In [ ]:
# transform the p-value results into a square matrix
anova_matrix = np.zeros((number_of_regions, number_of_regions))

for i in range(number_of_regions):
    for j in range(number_of_regions):
        anova_matrix[i,j] = results[i * number_of_regions + j]['PR(>F)'][0]

# Plot the p-values
plotting.plot_matrix(anova_matrix, labels=labels[1:], vmax=0.05, vmin=0, title='ANOVA p-values', cmap='hot')



# 5. Decoding

In [ ]:
# feature selection

# select feature with p-value < 0.05
selected_features = np.where(anova_matrix.flatten() < 0.05)

print(selected_features)

In [ ]:
from nilearn.maskers import NiftiMasker
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneGroupOut

from nilearn.decoding import Decoder
# “background”: Use this option if your images present a clear homogeneous background.
# “whole-brain-template”: This will extract the whole-brain part of your data by resampling 
# the MNI152 brain mask for your data’s field of view.


logo = LeaveOneGroupOut()

In [ ]:
c_func_runs = [os.path.join(funcdir_path, f) for f in func_runs]

In [ ]:
# Masking the data

from nilearn.masking import compute_epi_mask, compute_multi_epi_mask

mask_img = compute_multi_epi_mask(c_func_runs)

In [ ]:
# plot mask image
plot_img(mask_img)

In [ ]:
# Create the input matrix
X = np.array(list(partial_correlation_matrix.values()))

# Select the features
X = X[:, selected_features[0]]

In [ ]:
X.shape

In [ ]:
y = list(target_set.values())

# transform the target_set unique strings to integers
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

# 0 agaisnt all others
# y = np.where(y == 1, 0, 1)

print(y)


In [ ]:
runs_group = list(group_set.values())

print(runs_group)

# unique values in runs_group
print(np.unique(runs_group))

### Scikit CV

In [ ]:
from sklearn.model_selection import GroupKFold

# number of unique in runs_group
n_splits = len(np.unique(runs_group))

logo = LeaveOneGroupOut()





In [ ]:
from sklearn.metrics import roc_auc_score

def roc_auc_score_multiclass(actual_class, pred_class, average, multi_class):

  #creating a set of all the unique classes using the actual class list
  unique_class = set(actual_class)
  roc_auc_dict = {}
  for per_class in unique_class:
    #creating a list of all the classes except the current class 
    other_class = [x for x in unique_class if x != per_class]

    #marking the current class as 1 and all other classes as 0
    new_actual_class = [0 if x in other_class else 1 for x in actual_class]
    new_pred_class = [0 if x in other_class else 1 for x in pred_class]

    #using the sklearn metrics method to calculate the roc_auc_score
    roc_auc = roc_auc_score(new_actual_class, new_pred_class, average = average, multi_class = multi_class)
    roc_auc_dict[per_class] = roc_auc

  return roc_auc_dict

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score


clf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)

accuracy = {}
lr_roc_auc_multiclass  = {}
roc_vals_mc = {}
cm = {}

f = 0

for train, test in logo.split(X, y, groups=runs_group): 
    # print("%s %s" % (train, test))
    X_train, X_test = X[train], X[test]
    y_train, y_test = y[train], y[test]

    # print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

    clf.fit(X_train, y_train)

    # predict the labels
    y_pred = clf.predict(X_test)
    y_pred_prob = clf.predict_proba(X_test)

    # compute the accuracy
    accuracy[f] = np.mean(y_pred == y_test)

    # compute the confusion matrix
    cm[f] = confusion_matrix(y_test, y_pred)

    # compute the ROC
    roc_vals = roc_auc_score_multiclass(y_test, y_pred, average = 'micro', multi_class = 'ovr')


    lr_roc_auc_multiclass[f] = list(roc_vals.values())


    roc_vals_mc[f] = roc_auc_score(y_test, y_pred_prob, multi_class='ovr', average='weighted')

    f += 1
    
print(f'Accuracy: {accuracy}')
print(f'ROC AUC: {lr_roc_auc_multiclass}')

In [ ]:
roc_vals_mc

print(np.mean(list(roc_vals_mc.values())))

In [ ]:
# compute mean of the ROC AU
mean_roc_auc = np.mean(list(lr_roc_auc_multiclass.values()), axis=1)

print(f'Mean ROC AUC: {mean_roc_auc}')



# compute mean of the accuracy
mean_accuracy = (list(accuracy.values()))

print(f'Mean accuracy: {mean_accuracy}')




In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

import seaborn as sns

# plot confusion matrix totals
cm_total = np.zeros(cm[0].shape)

for i in range(len(cm)):
    cm_total += cm[i]



sns.heatmap(cm_total, annot= True, fmt='g', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_, )

plt.xlabel('Predicted')
plt.ylabel('True')


In [ ]:
from sklearn import svm, tree

#clf = svm.SVC(kernel='linear', C=1)
clf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)


# create cross validation using logo

scores = cross_val_score(clf, X, y, cv=logo, groups=runs_group,     scoring='roc_auc_ovr')

print(scores)

scores = cross_val_score(clf, X, y, cv=logo, groups=runs_group,     scoring='accuracy')

print(scores)


## 5.2 visualization of connectivity among classes

In [ ]:
# Visualizer the distribution of the scores
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(x=y, y = X[:,1])